In [ ]:
# !apt-get install p7zip-full

In [ ]:
import tensorflow as tf
print(tf.__version__)
#!pip uninstall tensorflow -2.15.0
#!pip install tensorflow==2.12.0

2.12.0


In [ ]:
# !7z x data.7z
print(tf.__version__)

2.12.0


# Imports

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, confusion_matrix, precision_recall_fscore_support, roc_auc_score
from tensorflow import keras
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Dense, Conv1D, Flatten, Conv2D
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import numpy as np
import seaborn as sns
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# import warnings

# warnings.filterwarnings('ignore')

In [ ]:
rawData = pd.read_csv('/content/drive/MyDrive/Power_Theft_Detection/Dataset/data.csv')
# rawData = rawData.iloc[:, -30:]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
rawData.columns

Index(['CONS_NO', 'FLAG', '2014/1/1', '2014/1/10', '2014/1/11', '2014/1/12',
       '2014/1/13', '2014/1/14', '2014/1/15', '2014/1/16',
       ...
       '2016/9/28', '2016/9/29', '2016/9/3', '2016/9/30', '2016/9/4',
       '2016/9/5', '2016/9/6', '2016/9/7', '2016/9/8', '2016/9/9'],
      dtype='object', length=1036)

# Data Processing

In [ ]:
rawData = pd.read_csv('/content/drive/MyDrive/Power_Theft_Detection/Dataset/data.csv')
rawData = rawData.iloc[:, :2].join(rawData.iloc[:, -30:])
rawData.columns.values[-30:] = [f"day{i}" for i in range(1, 31)]
infoData = pd.DataFrame()
infoData['FLAG'] = rawData['FLAG']
infoData['CONS_NO'] = rawData['CONS_NO']
data = rawData.drop(['FLAG', 'CONS_NO'], axis=1)

dropIndex = data[data.duplicated()].index  # duplicates drop
data = data.drop(dropIndex, axis=0)
infoData = infoData.drop(dropIndex, axis=0)

zeroIndex = data[(data.sum(axis=1) == 0)].index  # zero rows drop
data = data.drop(zeroIndex, axis=0)
infoData = infoData.drop(zeroIndex, axis=0)

# data.columns = pd.to_datetime(data.columns)  # columns reindexing according to dates
# data = data.reindex(sorted(data.columns), axis=1)
cols = data.columns

data.reset_index(inplace=True, drop=True)  # index sorting
infoData.reset_index(inplace=True, drop=True)

data = data.interpolate(method='linear', limit=2,  # filling NaN values
                        limit_direction='both', axis=0).fillna(0)

for i in range(data.shape[0]):  # outliers treatment
    m = data.loc[i].mean()
    st = data.loc[i].std()
    data.loc[i] = data.loc[i].mask(data.loc[i] > (m + 3 * st), other=m + 3 * st)

data.to_csv(r'visualization.csv', index=False, header=True)  # preprocessed data without scaling
# data.to_csv('/content/drive/MyDrive/python/visulaization.csv', index=False)

scale = MinMaxScaler()
scaled = scale.fit_transform(data.values.T).T
mData = pd.DataFrame(data=scaled, columns=data.columns)
print(mData)
preprData = pd.concat([infoData, mData], axis=1, sort=False)  # Back to initial format
print(preprData)
preprData.to_csv(r'preprocessedR.csv', index=False, header=True)


           day1      day2      day3      day4      day5      day6      day7  \
0      0.254098  0.631603  0.580146  0.486794  0.000000  0.140710  0.226321   
1      0.848848  1.000000  0.503687  0.877880  0.577880  0.308295  0.345161   
2      1.000000  0.619343  0.417855  0.528828  0.182889  0.171110  0.504030   
3      0.424419  0.781008  0.709302  0.906977  0.750000  0.534884  0.486434   
4      0.506466  0.086207  0.407328  0.092672  0.784483  0.196121  0.362069   
...         ...       ...       ...       ...       ...       ...       ...   
37119  0.672340  0.310638  1.000000  0.625532  0.140426  0.212766  0.497872   
37120  0.211538  0.503846  0.615385  1.000000  0.219231  0.473077  0.565385   
37121  0.661355  0.698805  0.157769  0.109960  0.002390  0.035060  0.003187   
37122  1.000000  0.894134  0.758937  0.706233  0.486251  0.492209  0.401925   
37123  0.668567  0.512473  0.279401  0.284391  0.395581  0.374911  0.253029   

           day8      day9     day10  ...     day21 

# Model Training

## Model Paramaneters

In [ ]:
tf.random.set_seed(1234)
epochs_number = 1  # number of epochs for the neural networks
test_set_size = 0.1  # percentage of the test size comparing to the whole dataset
oversampling_flag = 0  # set to 1 to over-sample the minority class
oversampling_percentage = 0.2  # percentage of the minority class after the oversampling comparing to majority class

## Data Loader

In [ ]:
# Definition of functions
def read_data():
    rawData = pd.read_csv('preprocessedR.csv')

    # Setting the target and dropping the unnecessary columns
    y = rawData[['FLAG']]
    X = rawData.drop(['FLAG', 'CONS_NO'], axis=1)

    print('Normal Consumers:                    ', y[y['FLAG'] == 0].count()[0])
    print('Consumers with Fraud:                ', y[y['FLAG'] == 1].count()[0])
    print('Total Consumers:                     ', y.shape[0])
    print("Classification assuming no fraud:     %.2f" % (y[y['FLAG'] == 0].count()[0] / y.shape[0] * 100), "%")

    # columns reindexing according to dates
    # X.columns = pd.to_datetime(X.columns)
    # X = X.reindex(X.columns, axis=1)

    # Splitting the dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y['FLAG'], test_size=test_set_size, random_state=0)
    print("Test set assuming no fraud:           %.2f" % (y_test[y_test == 0].count() / y_test.shape[0] * 100), "%\n")

    # Oversampling of minority class to encounter the imbalanced learning
    if oversampling_flag == 1:
        over = SMOTE(sampling_strategy=oversampling_percentage, random_state=0)
        X_train, y_train = over.fit_resample(X_train, y_train)
        print("Oversampling statistics in training set: ")
        print('Normal Consumers:                    ', y_train[y_train == 0].count())
        print('Consumers with Fraud:                ', y_train[y_train == 1].count())
        print("Total Consumers                      ", X_train.shape[0])

    return X_train, X_test, y_train, y_test

## Models

### Artificial Neural Network

In [ ]:
def ANN(X_train, X_test, y_train, y_test):
    print('Artificial Neural Network:')
    # for i in range(4,100,3):
    #     print("Epoch:",i)

    # Model creation
    model = Sequential()
    model.add(Dense(1000, input_dim=1034, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])

    # model.fit(X_train, y_train, validation_split=0, epochs=i, shuffle=True, verbose=0)
    model.fit(X_train, y_train, validation_split=0, epochs=epochs_number, shuffle=True, verbose=1)
    prediction =(model.predict(X_test) > 0.5).astype("int32")
    model.summary()
    results(y_test, prediction)

### 1D - Convolutional Neural Network

In [ ]:
def CNN1D(X_train, X_test, y_train, y_test):
    print('1D - Convolutional Neural Network:')

    # Transforming the dataset into tensors
    X_train = X_train.to_numpy().reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.to_numpy().reshape(X_test.shape[0], X_test.shape[1], 1)

    # Model creation
    model = Sequential()
    model.add(Conv1D(100, kernel_size=7, input_shape=(1034, 1), activation='relu'))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
    #model.load_weights('/content/drive/MyDrive/Power_Theft_Detection/model_weights.h5')

    # model.fit(X_train, y_train, epochs=1, validation_split=0.1, shuffle=False, verbose=1)
    model.fit(X_train, y_train, epochs=epochs_number, validation_split=0, shuffle=False, verbose=1)
    prediction = (model.predict(X_test) > 0.5).astype("int32")
    model.summary()
    results(y_test, prediction)
    return model

### 2D - Convolutional Neural Network

In [ ]:
def CNN2D(X_train, X_test, y_train, y_test):
    print('2D - Convolutional Neural Network:')

    # Transforming every row of the train set into a 2D array and then into a tensor
    n_array_X_train = X_train.to_numpy()
    n_array_X_train_extended = np.hstack((n_array_X_train, np.zeros(
        (n_array_X_train.shape[0], 2))))  # adding two empty columns in order to make the number of columns
    # an exact multiple of 7
    week = []
    for i in range(n_array_X_train_extended.shape[0]):
        a = np.reshape(n_array_X_train_extended[i], (-1, 7, 1))
        week.append(a)
    X_train_reshaped = np.array(week)

    # Transforming every row of the train set into a 2D array and then into a tensor
    n_array_X_test = X_test.to_numpy()  # X_test to 2D - array
    n_array_X_train_extended = np.hstack((n_array_X_test, np.zeros((n_array_X_test.shape[0], 2))))
    week2 = []
    for i in range(n_array_X_train_extended.shape[0]):
        b = np.reshape(n_array_X_train_extended[i], (-1, 7, 1))
        week2.append(b)
    X_test_reshaped = np.array(week2)

    input_shape = (1, 148, 7, 1)  # input shape of the tensor

    # Model creation
    model = Sequential()
    model.add(Conv2D(kernel_size=(7, 3), filters=32, input_shape=input_shape[1:], activation='relu',
                     data_format='channels_last'))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=keras.losses.binary_crossentropy,
                  optimizer='adam',
                  metrics=['accuracy'])
    # model.summary()
    #     model.fit(X_train_reshaped, y_train, validation_split=0.1, epochs=i, shuffle=False, verbose=0)
    model.fit(X_train_reshaped, y_train, validation_split=0.1, epochs=epochs_number, shuffle=False, verbose=1)

    # prediction = model.predict_classes(X_test)
    prediction = (model.predict(X_test_reshaped) > 0.5).astype("int32")
    model.summary()
    results(y_test, prediction)

### Logistic Regression

In [ ]:
def LR(X_train, X_test, y_train, y_test):
    print('Logistic Regression:')
    '''
    # Parameters selection
    param_grid = {'C': [0.1,1,10,100],'solver': ['newton-cg', 'lbfgs']}
    grid = GridSearchCV(LogisticRegression(max_iter=1000,random_state=0), param_grid=param_grid, n_jobs=-1)
    grid.fit(X_train, y_train)
    df = pd.DataFrame(grid.cv_results_)
    print(df[['param_C', 'param_solver', 'mean_test_score', 'rank_test_score']])
    '''
    model = LogisticRegression(C=1000, max_iter=1000, n_jobs=-1, solver='newton-cg')
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    results(y_test, prediction)

### Decision Tree

In [ ]:
def DT(X_train, X_test, y_train, y_test):
    print('Decision Tree:')
    model = DecisionTreeClassifier(random_state=0)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    results(y_test, prediction)

### Random Forest

In [ ]:
def RF(X_train, X_test, y_train, y_test):
    print('Random Forest:')
    '''
    # Parameters selection
    param_grid = {'n_estimators':[10,100,1000]}
    grid = GridSearchCV(RandomForestClassifier(random_state=0), param_grid=param_grid, n_jobs=-1)
    grid.fit(X_train, y_train)
    df = pd.DataFrame(grid.cv_results_)
    print(df[['param_criterion', 'mean_test_score', 'rank_test_score']])
    '''

    model = RandomForestClassifier(n_estimators=100, min_samples_leaf=1, max_features='auto',  # max_depth=10,
                                   random_state=0, n_jobs=-1)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    results(y_test, prediction)

### Support Vector Machine

In [ ]:
def SVM(X_train, X_test, y_train, y_test):
    model = SVC(random_state=0)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    results(y_test, prediction)

### XGBoost

In [ ]:
def XGB(X_train, X_test, y_train, y_test):
    # import XGBClassifier
    from xgboost import XGBClassifier

    # declare parameters
    params = {
                'objective':'binary:logistic',
                'max_depth': 4,
                'alpha': 10,
                'learning_rate': 1.0,
                'n_estimators':100
            }
    # instantiate the classifier
    model = XGBClassifier(**params)
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    results(y_test, prediction)
    return model
# fit the classifier to the training data
# xgb_clf.fit(X_train, y_train)

## Results

In [ ]:
def results(y_test, prediction):
    print("Accuracy", 100 * accuracy_score(y_test, prediction))
    print("RMSE:", mean_squared_error(y_test, prediction, squared=False))
    print("MAE:", mean_absolute_error(y_test, prediction))
    print("F1:", 100 * precision_recall_fscore_support(y_test, prediction)[2])
    print("AUC:", 100 * roc_auc_score(y_test, prediction))
    print(confusion_matrix(y_test, prediction), "\n")

# Main

In [ ]:
# ----Main----
X_train, X_test, y_train, y_test = read_data()

# Uncomment any model to test
# ANN(X_train, X_test, y_train, y_test)
# model = CNN1D(X_train, X_test, y_train, y_test)
# model = CNN2D(X_train, X_test, y_train, y_test)
# RF(X_train, X_test, y_train, y_test)
# LR(X_train, X_test, y_train, y_test)
# DT(X_train, X_test, y_train, y_test)
# SVM(X_train, X_test, y_train, y_test)
model = XGB(X_train, X_test, y_train, y_test)

Normal Consumers:                     33741
Consumers with Fraud:                 3383
Total Consumers:                      37124
Classification assuming no fraud:     90.89 %
Test set assuming no fraud:           90.98 %

Accuracy 90.51979531376246
RMSE: 0.30789941029884327
MAE: 0.09480204686237544
F1: [95.0070922   6.38297872]
AUC: 51.36179670033491
[[3349   29]
 [ 323   12]] 



In [ ]:
# Assuming 'model' is your trained Keras model
model.save_model('trained_model.h5')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [03:04:54] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


# **Prediction Functions**

## Process Input

In [ ]:
def preprocess_input(input_data):
    # rawData = input_data
    '''
    infoData = pd.DataFrame()
    infoData['FLAG'] = rawData['FLAG']
    infoData['CONS_NO'] = rawData['CONS_NO']
    data = rawData.drop(['FLAG', 'CONS_NO'], axis=1)
    '''
    data = input_data

    dropIndex = data[data.duplicated()].index  # duplicates drop
    data = data.drop(dropIndex, axis=0)
    # infoData = infoData.drop(dropIndex, axis=0)

    zeroIndex = data[(data.sum(axis=1) == 0)].index  # zero rows drop
    data = data.drop(zeroIndex, axis=0)
    # infoData = infoData.drop(zeroIndex, axis=0)

    # data.columns = pd.to_datetime(data.columns)  # columns reindexing according to dates
    # data = data.reindex(sorted(data.columns), axis=1)
    cols = data.columns

    data.reset_index(inplace=True, drop=True)  # index sorting
    # infoData.reset_index(inplace=True, drop=True)

    data = data.interpolate(method='linear', limit=2,  # filling NaN values
                            limit_direction='both', axis=0).fillna(0)

    for i in range(data.shape[0]):  # outliers treatment
        m = data.loc[i].mean()
        st = data.loc[i].std()
        data.loc[i] = data.loc[i].mask(data.loc[i] > (m + 3 * st), other=m + 3 * st)

    scale = MinMaxScaler()
    scaled = scale.fit_transform(data.values.T).T
    mData = pd.DataFrame(data=scaled, columns=data.columns)
    preprData = pd.concat([infoData, mData], axis=1, sort=False)  # Back to initial format


    # Setting the target and dropping the unnecessary columns
    y = preprData[['FLAG']]
    X = preprData.drop(['FLAG', 'CONS_NO'], axis=1)

    # columns reindexing according to dates
    # X.columns = pd.to_datetime(X.columns)
    # X = X.reindex(X.columns, axis=1)

    # Splitting the dataset into training set and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y['FLAG'], test_size=0.5, random_state=0)
    return  X_test, y_test

## Predict

In [ ]:
# Perform prediction
def predict(input_data, model):
    prediction = model.predict(xgb.DMatrix(input_data.values))
    binary_prediction = (prediction > 0.5).astype("int32")
    return binary_prediction

## Load Model

In [ ]:
import xgboost as xgb
print(xgb.__version__)
# Load the model
model = xgb.Booster()
model.load_model('trained_model.h5')

2.0.3


## Take sample inputs

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Power_Theft_Detection/Dataset/data.csv')

In [ ]:
df = df.iloc[:, -30:]

In [ ]:
df.columns.values[-30:] = [f"day{i}" for i in range(1, 31)]

In [ ]:
sampled_df = df.sample(n=3000)

In [ ]:
sampled_df.shape

(3000, 30)

In [ ]:
sampled_df.head()

,day1,day2,day3,day4,day5,day6,day7,day8,day9,day10,...,day21,day22,day23,day24,day25,day26,day27,day28,day29,day30
5711,5.33,4.79,10.98,11.64,6.85,6.36,7.06,7.63,8.58,6.28,...,5.33,7.33,3.79,11.06,8.72,8.88,5.14,5.62,5.69,5.45
37827,0.03,0.17,0.08,0.40,0.09,0.21,0.58,0.94,0.59,0.24,...,0.21,0.14,0.29,0.14,0.21,0.10,0.28,0.10,0.15,0.15
28972,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
15212,85.60,69.04,15.62,24.64,31.08,32.78,36.27,21.98,34.09,41.52,...,15.67,14.93,30.39,26.28,54.24,57.86,53.56,50.92,43.72,32.84
41447,5.32,5.74,5.59,4.82,4.82,5.58,4.34,5.36,4.12,4.67,...,4.94,3.78,5.53,4.58,5.54,5.39,5.67,6.06,5.63,5.44


## Generate Predictions

In [ ]:
inputData = pd.read_csv('/content/drive/MyDrive/Power_Theft_Detection/input_data.csv')
preprocessed_input, labels = preprocess_input(inputData)
predictions = predict(preprocessed_input, model)

In [ ]:
len(labels)

18562

In [ ]:
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int32)

In [ ]:
results(labels, predictions)

Accuracy 90.6852709837302
RMSE: 0.3052004098337648
MAE: 0.09314729016269799
F1: [95.11512926  0.        ]
AUC: 50.0
[[16833     0]
 [ 1729     0]] 



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## pickle file

In [ ]:
### create a pickle file using serialization
#import pickle
#pickle_out = open("classifier.pkl","wb")
#pickle.dump(classifier, pickle_out)
#pickle_out.close()